In [ ]:
import torch  # Modul pre efektívne vykonávanie výpočtov tenzorov
import torch.nn as nn  # Modul pre vytváranie a prácu s neurónovými sieťami
import torch.optim as optim  # Modul pre optimalizačné algoritmy, ako napr. Adam
from torchvision import transforms  # Transformácie pre spracovanie obrázkov
from torch.utils.data import DataLoader, Dataset  # Nástroje na prácu s datasetmi a dávkami (batchmi)
from PIL import Image  # Načítanie a spracovanie obrázkov
import os  # Modul pre prácu so súbormi a priečinkami
import numpy as np  # Modul na numerické výpočty
import matplotlib.pyplot as plt  # Modul na vizualizáciu obrázkov a grafov
from google.colab import files  # Modul pre nahrávanie súborov do Google Colab

# Definícia datasetu s pridaným šumom
class NoisyImageDataset(Dataset):   # Trieda pre vlastný dataset
    def __init__(self, image_paths, transform, noise_level=0.1):
        self.image_paths = image_paths  # Cesty k obrázkom
        self.transform = transform  # Transformácie obrázkov
        self.noise_level = noise_level  # Úroveň pridávaného šumu

    def __len__(self):  # Počet obrázkov v datasete
        return len(self.image_paths)

    def __getitem__(self, idx):  # Získanie obrázka a pridanie šumu
        img_path = self.image_paths[idx]  # Cesta k obrázku
        clean_image = Image.open(img_path).convert("RGB")  # Načítanie obrázka a konverzia na RGB
        clean_image = self.transform(clean_image)  # Použitie transformácií

        # Pridanie náhodného šumu
        noise = torch.randn_like(clean_image) * self.noise_level  # Generovanie náhodného šumu
        noisy_image = clean_image + noise  # Pridanie šumu k čistému obrázku
        noisy_image = torch.clamp(noisy_image, 0., 1.)  # Obmedzenie hodnôt na rozsah [0, 1]

        return noisy_image, clean_image  # Vráti zašumený a čistý obrázok


# Nahrávanie obrázkov do Google Colab
uploaded = files.upload()  # Nahranie obrázkov prostredníctvom užívateľského rozhrania
image_paths = list(uploaded.keys())  # Získanie zoznamu názvov nahraných súborov

# Definícia transformácií pre obrázky
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Zmena veľkosti na 256x256 pixelov
    transforms.ToTensor()  # Konverzia obrázka na tensor
])

# Načítanie datasetu s obrázkami a šumom
noise_level = 0.2  # Úroveň pridávaného šumu
dataset = NoisyImageDataset(image_paths, transform, noise_level=noise_level)

# Načítanie datasetu do DataLoadera
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)  # Rozdelenie na dávky (batch size = 16)

# Definícia CNN modelu pre odstránenie šumu
class DenoisingCNN(nn.Module):  # Trieda pre definíciu modelu
    def __init__(self):
        super(DenoisingCNN, self).__init__()
        # Encoder (časť modelu na extrakciu čŕt z obrázku)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Konvolučná vrstva: 3 vstupné kanály (RGB), 64 výstupných
            nn.ReLU(),  # Aktivácia ReLU
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Druhá konvolučná vrstva
            nn.ReLU()  # Aktivácia ReLU
        )
        # Decoder (časť modelu na rekonštrukciu obrázka)
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Konvolučná vrstva
            nn.ReLU(),  # Aktivácia ReLU
            nn.Conv2d(64, 3, kernel_size=3, padding=1),  # Výstupná vrstva: rekonštrukcia obrázka s 3 kanálmi
            nn.Sigmoid()  # Aktivácia Sigmoid pre normalizáciu na rozsah [0, 1]
        )

    def forward(self, x):  # Forward pass (prenos vstupu cez model)
        x = self.encoder(x)  # Prechod cez encoder
        x = self.decoder(x)  # Prechod cez decoder
        return x  # Návrat výstupu


# Inicializácia modelu, funkcie straty a optimalizátora
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Použitie GPU, ak je dostupné
model = DenoisingCNN().to(device)  # Presun modelu na zariadenie (CPU/GPU)
criterion = nn.MSELoss()  # Funkcia straty: stredná kvadratická chyba (MSE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Optimalizátor Adam s učebnou rýchlosťou 0.001

# Tréningový cyklus
num_epochs = 100  # Počet epôch
for epoch in range(num_epochs):
    model.train()  # Nastavenie modelu do tréningového režimu
    train_loss = 0.0  # Počiatočná hodnota straty
    for noisy_images, clean_images in data_loader:  # Iterácia cez dávky dát
        noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)  # Presun dát na zariadenie

        # Forward pass (predikcia výstupu)
        outputs = model(noisy_images)
        loss = criterion(outputs, clean_images)  # Výpočet straty medzi predikciou a čistými obrázkami

        # Backward pass (aktualizácia parametrov)
        optimizer.zero_grad()  # Nulovanie gradientov
        loss.backward()  # Výpočet gradientov
        optimizer.step()  # Aktualizácia parametrov modelu

        train_loss += loss.item() * noisy_images.size(0)  # Akumulácia straty

    train_loss /= len(data_loader.dataset)  # Priemerná strata pre epochu
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")  # Výpis straty po každej epoche

# Vizualizácia výsledkov
model.eval()  # Nastavenie modelu do evaluačného režimu
with torch.no_grad():  # Vypnutie výpočtu gradientov
    for noisy_images, clean_images in data_loader:
        noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)  # Presun dát na zariadenie
        outputs = model(noisy_images)  # Predikcia modelu

        # Konverzia tensorov na numpy polia pre vizualizáciu
        noisy = noisy_images[0].cpu().numpy().transpose(1, 2, 0)
        clean = clean_images[0].cpu().numpy().transpose(1, 2, 0)
        denoised = outputs[0].cpu().numpy().transpose(1, 2, 0)

        # Zobrazenie obrázkov
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title("Noisy Image")  # Zašumený obrázok
        plt.imshow(noisy)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title("Denoised Image")  # Obrázok po odstránení šumu
        plt.imshow(denoised)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title("Clean Image")  # Čistý obrázok
        plt.imshow(clean)
        plt.axis('off')

        plt.show()
        break  # Zobraziť len jednu dávku obrázkov

# Uloženie trénovaného modelu
torch.save(model.state_dict(), 'denoising_cnn.pth')  # Uloženie parametrov modelu do súboru
